# Effect of UBI on labor supply in OG-USA small open economy

Testing the income effect.

Goals:

* What's the implied income effect when
* How does this vary with frisch and epsilon?
* How does this compare to microeconometric studies on the income effect?
* How does it compare to other models like CBO and PWBM?
* How does it vary with lifetime income level and age?

Intermediate questions:
* What is the range of the implied UBI based on its ETR effect? Is it roughly the `taxcalc`-supplied UBI value multiplied by the number of people per tax unit?
* What's the overall labor response to the \$1,000 UBI across lifetime income groups and age?

## Setup

In [41]:
import pandas as pd
import numpy as np
import ogusa
import plotly.express as px

In [114]:
base = pd.read_pickle('OUTPUT_BASELINE/SS/SS_vars.pkl')
reform = pd.read_pickle('OUTPUT_REFORM/SS/SS_vars.pkl')
reform_params = pd.read_pickle('OUTPUT_REFORM/model_params.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'OUTPUT_BASE/model_params.pkl'

## Show labor responses

As a precursor to elasticities. These are in response to a \$1,000-per-year-per-person (including children) UBI.

In [102]:
nss_diff = pd.DataFrame(reform['nssmat'] / base['nssmat'] - 1)\
    .reset_index().melt('index')
nss_diff.columns = ['age', 'skill', 'n_diff']
nss_diff['n_diff_pct'] = (nss_diff.n_diff * 100).round(2)
nss_diff.skill = nss_diff.skill.astype(int)
# Age starts at 21.
nss_diff.age += 21

In [106]:
fig = px.line(nss_diff, x='age', y='n_diff_pct', color='skill',
              color_discrete_sequence=px.colors.sequential.Viridis)
fig.update_layout(
        title='Change to labor supply from $1,000 UBI in small open economy',
        xaxis_title='Age',
        yaxis_title='Change to labor supply',
        yaxis_ticksuffix='%',
        font=dict(family='Roboto'),
        hovermode='x',
        plot_bgcolor='white',
        legend_title_text='Lifetime income group'
    )
fig.update_traces(hovertemplate=None)

**TODO:** Compute overall response by lifetime income group by weighting by $\omega$, and overall response by age by weighting by $\lambda$.

In [123]:
omegas_ss = reform_params.omega[-1, :].sum()  # Final time period.

## Other exploratory

Change between base and reform to `yss_before_tax` matches change to `nss` in the first period,
diverges after that.

In [79]:
assert np.allclose(reform['yss_before_tax_mat'][0, :] /
                   base['yss_before_tax_mat'][0, :],
                   reform['nssmat'][0, :] /
                   base['nssmat'][0, :])

In [80]:
assert not np.allclose(reform['yss_before_tax_mat'][1:, :] /
                       base['yss_before_tax_mat'][1:, :],
                       reform['nssmat'][1:, :] /
                       base['nssmat'][1:, :])

## Compute elasticities

### Compute change in income from the UBI

Considering each group's income from baseline labor supply, tax policy, and other conditions other than ETR (which is affected only by the UBI).

```
def net_taxes(r, w, b, n, bq, factor, tr, theta, t, j, shift, method,
              e, etr_params, p):
```

In [24]:
assert np.allclose(base['rss'], reform['rss'])
assert np.allclose(base['wss'], reform['wss'])
assert np.allclose(base['factor_ss'], reform['factor_ss'])

In [29]:
# r, w, and factor are the same between base and reform.
# Label others as _base or _reform depending on what's
# needed for the UBI calculation.
r = base['rss']
w = base['wss']
# Label other variables as base.
# _s indicates for age s, vs. _splus1.
b_s_base = base['bssmat_s']  # Savings.
n_base = base['nssmat']  # Labor supply.
bq_base = base['bqssmat']  # Bequests received.
factor = base['factor_ss']  # Scaling factor from units to dollars.
tr_base = base['TR_ss']  # Government transfers to the household.
theta_base = base['theta']  # Social security replacement rate value.

Specifications can come from baseline.

In [47]:
p = ogusa.Specifications(baseline=True, time_path=False)

This is the calculation that breaks the `net_taxes` function call below. All dimensions seem to align, so what's the problem?

In [124]:
r * b_s_base + w * n_base * p.e

array([[0.18501207, 0.11210508, 0.11318395, 0.13045544, 0.16156734,
        0.26797914, 0.6714108 ],
       [0.17929378, 0.11748347, 0.12401859, 0.1449218 , 0.1792005 ,
        0.28404022, 0.72131939],
       [0.1938584 , 0.1370046 , 0.15063277, 0.17829   , 0.22018204,
        0.33542504, 0.85955514],
       [0.18259575, 0.13836818, 0.15777147, 0.18895491, 0.23316828,
        0.34367958, 0.89180468],
       [0.18575358, 0.15029473, 0.17711234, 0.21442302, 0.26440403,
        0.3790937 , 0.99399675],
       [0.18709881, 0.16096518, 0.19538014, 0.23884534, 0.29426434,
        0.41227132, 1.08961329],
       [0.18735782, 0.17074505, 0.21281771, 0.26242981, 0.32298511,
        0.44395012, 1.17868367],
       [0.1883307 , 0.18113419, 0.23118491, 0.28732406, 0.3532726 ,
        0.4783956 , 1.27494329],
       [0.18922695, 0.19140826, 0.24953116, 0.31235649, 0.38372083,
        0.51393925, 1.3754733 ],
       [0.186827  , 0.19803696, 0.263046  , 0.33141211, 0.4068152 ,
        0.54080273, 1.4

In [63]:
j = 0  # Try a list or None later to get all lifetime skill levels.
net_tax_ubi = ogusa.tax.net_taxes(
    r=r, w=w, b=b_s_base, n=n_base, bq=bq_base, factor=factor,
    tr=tr_base, theta=theta_base,
    t=None,
    j=j,
    shift=False,
    method='SS',
    e=p.e[:, j],
    etr_params=reform_params.etr_params[-1, :, :],
    p=p)

ValueError: operands could not be broadcast together with shapes (80,) (80,7) 

In [125]:
net_tax_ubi = ogusa.tax.net_taxes(
    r=r, w=w, b=b_s_base, n=n_base, bq=bq_base, factor=factor,
    tr=tr_base, theta=theta_base,
    t=None,
    j=None,
    shift=False,
    method='SS',
    e=p.e,
    etr_params=reform_params.etr_params[-1, :, :],
    p=p)

ValueError: operands could not be broadcast together with shapes (80,) (80,7) 